# Testing models

----


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-config" data-toc-modified-id="Set-config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set config</a></span></li><li><span><a href="#Collect-cases" data-toc-modified-id="Collect-cases-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Collect cases</a></span></li><li><span><a href="#Treat-databases" data-toc-modified-id="Treat-databases-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Treat databases</a></span></li><li><span><a href="#Run-model" data-toc-modified-id="Run-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run model</a></span><ul class="toc-item"><li><span><a href="#Testing-plot" data-toc-modified-id="Testing-plot-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Testing plot</a></span></li></ul></li><li><span><a href="#Compare-hospital-capacity" data-toc-modified-id="Compare-hospital-capacity-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compare hospital capacity</a></span></li><li><span><a href="#Draft" data-toc-modified-id="Draft-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Draft</a></span></li></ul></div>

## Set config

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import yaml
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.dates as md

from paths import *
from scripts import get_data, clean_data

%reload_ext autoreload
%autoreload 2

config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)

## Collect cases

In [2]:
config['get_data_paths']

{'cases_covid19br': 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv',
 'cases_brasilio': 'https://brasil.io/api/dataset/covid19/caso/data?format=json'}

In [3]:
df_cases_covid19br = get_data.city_cases_covid19br(config['get_data_paths']['cases_covid19br'])
df_cases_brasilio = get_data.city_cases_brasilio(config['get_data_paths']['cases_brasilio'])

In [4]:
df_cases_covid19br

,date,country,state,city,newCases,totalCases,last_update
583,2020-03-20,Brazil,TOTAL,TOTAL,318,1012,2020-03-23 11:12:32.723969
506,2020-03-20,Brazil,SC,Chapecó/SC,1,1,2020-03-23 11:12:32.723969
492,2020-03-20,Brazil,RS,Torres/RS,1,2,2020-03-23 11:12:32.723969
493,2020-03-20,Brazil,BA,Lauro de Freitas/BA,1,3,2020-03-23 11:12:32.723969
494,2020-03-20,Brazil,RS,Alvorada/RS,1,2,2020-03-23 11:12:32.723969
495,2020-03-20,Brazil,RS,Erechim/RS,1,3,2020-03-23 11:12:32.723969
496,2020-03-20,Brazil,RS,Santana do Livramento/RS,1,2,2020-03-23 11:12:32.723969
497,2020-03-20,Brazil,PI,Teresina/PI,1,4,2020-03-23 11:12:32.723969
498,2020-03-20,Brazil,CE,Fortim/CE,1,1,2020-03-23 11:12:32.723969
499,2020-03-20,Brazil,AP,Macapá/AP,1,1,2020-03-23 11:12:32.723969


In [5]:
df_cases_brasilio

,city,city_ibge_code,confirmed,confirmed_per_100k_inhabitants,date,death_rate,deaths,estimated_population_2019,is_last,place_type,state,last_update
0,Rio Branco,401.0,11,2.70059,2020-03-22,NaN,0,407319.0,True,city,AC,2020-03-23 11:12:38.110144
1,None,12.0,11,1.24726,2020-03-22,NaN,0,881935.0,True,state,AC,2020-03-23 11:12:38.110144
2,Maceió,4302.0,7,0.68698,2020-03-22,NaN,0,1018948.0,True,city,AL,2020-03-23 11:12:38.110144
3,None,27.0,7,0.20975,2020-03-22,NaN,0,3337357.0,True,state,AL,2020-03-23 11:12:38.110144
4,Macapá,303.0,1,0.19868,2020-03-22,NaN,0,503327.0,True,city,AP,2020-03-23 11:12:38.110144
5,None,16.0,1,0.11824,2020-03-22,NaN,0,845731.0,True,state,AP,2020-03-23 11:12:38.110144
6,Barreiras,3201.0,1,0.64334,2020-03-22,NaN,0,155439.0,True,city,BA,2020-03-23 11:12:38.110144
7,Camaçari,5701.0,1,0.33430,2020-03-22,NaN,0,299132.0,True,city,BA,2020-03-23 11:12:38.110144
8,Conceição do Jacuípe,8507.0,1,3.01632,2020-03-22,NaN,0,33153.0,True,city,BA,2020-03-23 11:12:38.110144
9,Feira de Santana,10800.0,6,0.97581,2020-03-22,NaN,0,614872.0,True,city,BA,2020-03-23 11:12:38.110144


## Treat databases

In [6]:
config['raw_paths'], config['treated_paths']

({'cases_covid19br': 'city_cases_covid19br.csv',
  'cases_brasilio': 'city_cases_brasilio.csv',
  'sus': 'covid19_SUS_database.csv'},
 {'cases_covid19br': 'last_cases_covid19br.csv',
  'cases_brasilio': 'last_cases_covid19br.csv',
  'sus': 'treated_covid19_SUS_database.csv'})

In [21]:
df_cases_covid19br = clean_data.treat_covid19br(config['raw_paths']['cases_covid19br'], config['treated_paths']['cases_covid19br'])
df_cases_brasilio = clean_data.treat_brasilio(config['raw_paths']['cases_brasilio'], config['treated_paths']['cases_brasilio']) 

In [8]:
df_cases_covid19br.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 34 to 122
Data columns (total 5 columns):
region_id     122 non-null object
city          122 non-null object
place_type    122 non-null object
date          122 non-null object
confirmed     122 non-null int64
dtypes: int64(1), object(4)
memory usage: 5.7+ KB


In [9]:
df_cases_brasilio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 352 to 446
Data columns (total 13 columns):
city                              143 non-null object
city_ibge_code                    142 non-null float64
confirmed                         143 non-null int64
confirmed_per_100k_inhabitants    142 non-null float64
date                              143 non-null object
death_rate                        4 non-null float64
deaths                            143 non-null int64
estimated_population_2019         142 non-null float64
is_last                           143 non-null bool
place_type                        143 non-null object
state                             143 non-null object
last_update                       143 non-null object
region_id                         143 non-null object
dtypes: bool(1), float64(4), int64(2), object(6)
memory usage: 14.7+ KB


In [10]:
print('Total cases covid19br:', df_cases_covid19br['confirmed'].sum())
print('Total cases brasilio:', df_cases_brasilio[df_cases_brasilio['place_type'] != 'state']['confirmed'].sum())

Total cases covid19br: 1012
Total cases brasilio: 1265


In [158]:
df_sus = clean_data.treat_sus(config['raw_paths']['sus'], config['treated_paths']['sus'])
df_sus

,region_id,municipio,uf,populacao,quantidade_leitos,ventiladores_disponiveis
0,AC,AC,AC,881935,1484.0,8.0
1,AL,AL,AL,3337357,5891.0,39.0
2,AM,AM,AM,4144597,5700.0,58.0
3,AP,AP,AP,845731,1098.0,6.0
4,BA,BA,BA,14873064,28960.0,147.0
5,CE,CE,CE,9132078,18510.0,130.0
6,DF,DF,DF,3015268,6705.0,112.0
7,ES,ES,ES,4018650,7929.0,101.0
8,GO,GO,GO,7018354,17445.0,38.0
9,MA,MA,MA,7075181,13778.0,73.0


## Run model

Running SIR model with different level of output.

From [Modeling COVID-19 Spread vs Healthcare Capacity
](https://alhill.shinyapps.io/COVID19seir/):

>* Mild infection (I1) - These individuals have symptoms like fever and cough and may have mild pneumonia. **Hospitalization is not required** (though in many countries such individuals are also hospitalized)

>* Severe infection (I2) - These individuals have more severe pneumonia that leads to dyspnea, respiratory frequency <30/min, blood oxygen saturation <93%, partial pressure of arterial oxygen to fraction of inspired oxygen ratio <300, and/or lung infiltrates >50% within 24 to 48 hours. **Hospitalization and supplemental oxygen are generally required.**

>* Critical infection (I3) - These individuals experience respiratory failure, septic shock, and/or multiple organ dysfunction or failure. **Treatment in an ICU, often with mechanical ventilation, is required.**


In [159]:
# merge sus and cases data
df = pd.merge(df_cases_brasilio, df_sus, how='left', on='region_id')
df

,city,city_ibge_code,confirmed,confirmed_per_100k_inhabitants,date,death_rate,deaths,estimated_population_2019,is_last,place_type,state,last_update,region_id,municipio,uf,populacao,quantidade_leitos,ventiladores_disponiveis
0,São Paulo,50308.0,358,2.92197,2020-03-20,0.0251,9,12252023.0,True,city,SP,2020-03-23 11:12:38.110144,SAO PAULO SP,São Paulo,SP,12252023.0,27847.0,276.0
1,Rio de Janeiro,4557.0,168,2.50041,2020-03-22,NaN,0,6718903.0,True,city,RJ,2020-03-23 11:12:38.110144,RIO DE JANEIRO RJ,Rio de Janeiro,RJ,6718903.0,13933.0,377.0
2,Brasília,108.0,117,3.88025,2020-03-22,NaN,0,3015268.0,True,city,DF,2020-03-23 11:12:38.110144,BRASILIA DF,Brasília,DF,3015268.0,6705.0,112.0
3,Fortaleza,4400.0,116,4.34564,2020-03-22,NaN,0,2669342.0,True,city,CE,2020-03-23 11:12:38.110144,FORTALEZA CE,Fortaleza,CE,2669342.0,8287.0,40.0
4,Porto Alegre,14902.0,35,2.35885,2020-03-21,NaN,0,1483771.0,True,city,RS,2020-03-23 11:12:38.110144,PORTO ALEGRE RS,Porto Alegre,RS,1483771.0,7019.0,154.0
5,Salvador,27408.0,33,1.14889,2020-03-22,NaN,0,2872347.0,True,city,BA,2020-03-23 11:12:38.110144,SALVADOR BA,Salvador,BA,2872347.0,8171.0,61.0
6,Curitiba,6902.0,31,1.60364,2020-03-22,NaN,0,1933105.0,True,city,PR,2020-03-23 11:12:38.110144,CURITIBA PR,Curitiba,PR,1933105.0,5623.0,81.0
7,Belo Horizonte,6200.0,30,1.19423,2020-03-21,NaN,0,2512070.0,True,city,MG,2020-03-23 11:12:38.110144,BELO HORIZONTE MG,Belo Horizonte,MG,2512070.0,8730.0,63.0
8,Recife,11606.0,28,1.70138,2020-03-22,NaN,0,1645727.0,True,city,PE,2020-03-23 11:12:38.110144,RECIFE PE,Recife,PE,1645727.0,8328.0,121.0
9,Campo Grande,2704.0,19,2.12058,2020-03-22,NaN,0,895982.0,True,city,MS,2020-03-23 11:12:38.110144,CAMPO GRANDE MS,Campo Grande,MS,895982.0,2220.0,28.0


In [19]:
from models import sir

In [86]:
df_final = pd.DataFrame()

for region_id in df['region_id'].unique():
    for scenario, s_values  in config['scenarios'].items():
        for bound, r0 in s_values['R0'].items():
            
            df_city = df.query(f"region_id == '{region_id}'")

            current_state = {
                'population': df_city['populacao'].values[0],
                'current_infected': df_city['confirmed'].values[0]
            }

            config['model_parameters']['sir']['R0'] = r0

            res = sir.entrypoint(current_state, config['model_parameters']['sir'])
            
            res['region_id'] = region_id
            res['scenario'] = scenario
            res['bound'] = bound
            
            df_final = pd.concat([df_final, res], axis=0)

# df_final.to_csv(OUTPUT_PATH / 'model_for_city_{}.csv'.format(study), index=False)
# df_final.to_csv('../data/output/model_for_city.csv', index=False)

In [87]:
df_final['municipio'] = df_final['region_id'].apply(lambda x: x[:-2])
df_final['uf'] = df_final['region_id'].apply(lambda x: x[-2:])

df_final

,S,R,I,days,I2,I3,I1,region_id,scenario,bound,municipio,uf
0,1.225166e+07,358.000000,0.000000,0.0,0.000000,0.000000,0.000000,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
1,1.225160e+07,395.649096,25.100727,1.0,3.765109,1.255036,20.080581,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
2,1.225153e+07,437.257169,52.841161,2.0,7.926174,2.642058,42.272929,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
3,1.225146e+07,483.240438,83.498877,3.0,12.524832,4.174944,66.799102,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
4,1.225137e+07,534.058859,117.380632,4.0,17.607095,5.869032,93.904505,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
5,1.225128e+07,590.220719,154.825429,5.0,23.223814,7.741271,123.860343,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
6,1.225117e+07,652.287712,196.207909,6.0,29.431186,9.810395,156.966327,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
7,1.225106e+07,720.880538,241.942094,7.0,36.291314,12.097105,193.553676,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
8,1.225093e+07,796.685101,292.485524,8.0,43.872829,14.624276,233.988419,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP
9,1.225079e+07,880.459337,348.343821,9.0,52.251573,17.417191,278.675057,SAO PAULO SP,nothing,lower_bound,SAO PAULO,SP


In [88]:
df_final.to_csv(OUTPUT_PATH / 'model_for_city.csv', index=False)

### Testing plot

In [89]:
# sp = df_final[df_final['region_id'] == 'SAO PAULO SP']
# sp['days'] = sp.index
# sp.head()

In [90]:
# fig = px.line(sp, x='days', y='I', color='scenario')
# fig

## Compare hospital capacity

In [162]:
args_dday = {'critical': {'infected': 'I3', 'resource': 'ventiladores_disponiveis'}, 
             'severe': {'infected': 'I2', 'resource': 'quantidade_leitos'}}

In [179]:
def calculate_dday(df_sus, df_final, args):
    
    cols = ['region_id', 'scenario', 'days'] + [args['infected'], args['resource']]
    
    df = df_sus.merge(df_final, on='region_id')[cols]
    
    df = df[df[args['resource']] < df[args['infected']]] # days with overload
    dday = df.sort_values(['region_id', 'days']).drop_duplicates('region_id', keep='first') # dday: first overload day
    dday = dday.rename({'days': 'dday_{}'.format(args['infected'])}, axis=1)
    
    return dday

In [180]:
dday_critical = calculate_dday(df_sus, df_final, args_dday['critical'])
dday_critical

,region_id,scenario,dday_I3,I3,ventiladores_disponiveis
390,AC,nothing,24.0,8.192741,8.0
239365,ALVORADA RS,nothing,1.0,0.007011,0.0
296848,ANAPOLIS GO,nothing,22.0,1.099660,1.0
298657,APARECIDA DE GOIANIA GO,nothing,1.0,0.003506,0.0
17569,AQUIRAZ CE,nothing,1.0,0.017528,0.0
48710,ARACAJU SE,nothing,32.0,17.195768,16.0
241950,BAGE RS,nothing,24.0,2.233480,2.0
199837,BALNEARIO CAMBORIU SC,nothing,1.0,0.021034,0.0
114584,BARRA MANSA RJ,nothing,26.0,1.006600,1.0
54901,BARREIRAS BA,nothing,1.0,0.003506,0.0


In [181]:
dday_severe = calculate_dday(df_sus, df_final, args_dday['severe'])
dday_severe

,region_id,scenario,dday_I2,I2,quantidade_leitos
418,AC,nothing,52.0,1498.179545,1484.0
239775,ALVORADA RS,nothing,45.0,99.374739,97.0
296888,ANAPOLIS GO,nothing,62.0,1157.273150,1012.0
299089,APARECIDA DE GOIANIA GO,nothing,67.0,1221.663618,1104.0
17975,AQUIRAZ CE,nothing,41.0,135.447601,123.0
48736,ARACAJU SE,nothing,58.0,2237.775289,2161.0
241974,BAGE RS,nothing,48.0,226.560290,220.0
200246,BALNEARIO CAMBORIU SC,nothing,44.0,252.240139,248.0
114616,BARRA MANSA RJ,nothing,58.0,328.030934,296.0
55326,BARREIRAS BA,nothing,60.0,432.931586,379.0


In [155]:
t = dday[dday['municipio'] != dday['uf']]
px.histogram(t, 'days', title='Número de cidades por qtd de dias para colapso hospitalar (sem intervenção)')

In [156]:
t.min()

region_id            ALVORADA RS
municipio               Alvorada
uf                            AC
populacao                   2878
quantidade_leitos              0
scenario                 nothing
days                           1
I3                    0.00350567
dtype: object

In [157]:
len(df_capacity['region_id'].unique()) # All cities/states reach max capacity doing nothing

142

## Draft

In [28]:
# def model(current_state, model_paramenters):
    
#     current_state = {
#         'population'
#         'current_infected'
#     }
    
#     model_parameters = {
#         'R0'
#         'sick_days'
#         'i2_percentage'
#         'i3_percentage'
#     }
    
#     return {
#         'day'
#         'S'
#         'R'
#         'I1'
#         'I2'
#         'I3'
#     }

In [ ]:
# # UTI demand
# uti_demand = I * perc_infectados_internacao / 100

# # Create output dataframe
# dd = pd.DataFrame([municipio]*days,columns=['municipio'])

# dd['uf']=df_city['uf'].values[0]
# dd['place_type'] = df_city['place_type'].values[0]
# dd['populacao'] = N
# dd['dias_doente'] = dias_doente
# dd['percentual_infectados_que_precisam_de_internacao'] = perc_infectados_internacao
# dd['data_numero_infectados'] = df_city['date'].values[0]
# dd['numero_infectados'] = I_i0

# n_beds = df_city['quantidade_leitos'].values[0]
# dd['leitos_total'] = n_beds

# dd['dias'] = t
# dd['demanda_por_uti'] = uti_demand
# dd['cenario'] = scenario
# dd['suscetiveis'] = S
# dd['infectados'] = I
# dd['recuperados'] = R


# return dd

In [138]:
# municipio = 'São Paulo'
# mask = df_final['municipio']== municipio
# dd = df_final[mask]

# perc = 80
# fig = px.line(dd, x='dias',y='demanda_por_uti', color='cenario')
# fig

In [12]:
# df_final = pd.DataFrame()

# for study in parameters_statics.keys():
#     dic_study = parameters_statics[study]
    
#     for municipio in df['municipio'].unique():
#         for scenario in dic_study.keys():

#             mask = df['municipio'] == municipio
#             df_city = df[mask]
            
#             dd = sir_results(df_city, scenario, dias_doente, perc_infectados_internacao, study)
#             df_final = pd.concat([df_final,dd],axis=0)

#     df_final.to_csv(OUTPUT_PATH / 'model_for_city_{}.csv'.format(study), index=False)
        
# # df_final.to_csv('../data/output/model_for_city.csv', index=False)

- MODELO (antigo)

  - SIR - https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/
  - https://colab.research.google.com/drive/1bmR-W2NvQ7gPnldTfei927NtGJUCC1BB#scrollTo=47Er1T5YZBXf

    ro - fixo para cada cidade
    populacao de cada cidade